In [3]:
import numpy as np
import pandas as pd

from pathlib import Path


# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [29]:
# read dataset from csv file
base_dir = Path.cwd().parent
data_file_path = base_dir/"data"/"adult.csv"
absolute_data_file_path = data_file_path.resolve()

data = pd.read_csv(absolute_data_file_path)
# print(data.head())
# data.info()

# column_data = data["age"]
# column_data.describe()
# column_data.mean()
# column_data.median()
# column_data.sum()
# column_data.value_counts()
# column_data.unique()
# column_data.isnull().sum()

contains_symbol = data.map(lambda x: "?" in str(x))
rows_with_symbol = data[contains_symbol.any(axis=1)]
print(rows_with_symbol)

# remove all rows, which contains ?



       Unnamed: 0  age workclass     education         marital-status  \
14             14   40   Private     Assoc-voc     Married-civ-spouse   
27             27   54         ?  Some-college     Married-civ-spouse   
38             38   31   Private  Some-college     Married-civ-spouse   
51             51   18   Private       HS-grad          Never-married   
61             61   32         ?       7th-8th  Married-spouse-absent   
...           ...  ...       ...           ...                    ...   
32530       32530   35         ?     Bachelors     Married-civ-spouse   
32531       32531   30         ?     Bachelors          Never-married   
32539       32539   71         ?     Doctorate     Married-civ-spouse   
32541       32541   41         ?       HS-grad              Separated   
32542       32542   72         ?       HS-grad     Married-civ-spouse   

          occupation   relationship                race     sex  \
14      Craft-repair        Husband  Asian-Pac-Islander 

# Task 1
Print the count of men and women in the dataset.

In [1]:
# write your code for task 1 here

# Task 2
Find the average age of men in dataset

In [ ]:
# write your code for task 2 here

# Task 3
Get the percentage of people from Poland (native-country)

In [ ]:
# write your code for task 3 here

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [2]:
# write your code for task 4 here

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [3]:
# write your code for task 5 here

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [ ]:
# write your code for task 6 here

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [5]:
# write your code for task 7 here

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [6]:
# write your code for task 8 here

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [7]:
# write your code for task 9 here